In [1]:
import panel as pn;     pn.extension('plotly', 'katex', 'mathjax')
import plotly.graph_objects as go
import numpy as np
import sympy as sp

<div style="float:center;width:100%;text-align: center;">
    <strong style="height:60px;color:darkred;font-size:40px;">Drawing Vectors and Planes in 3D</strong><br>
</div>

# 1. Basic Drawing Routines

## 1.1 Implementation

In [2]:
def create_fig( traces,x_range=(-10, 10), y_range=(-10, 10), z_range=(-10,10), camera_position=None, width=700 ):
    ''' Create a plotly figure'''
    fig = go.Figure(data=traces)

    # Set the layout of the figure, including the camera position if specified
    camera = dict(eye=dict(x=-1.25, y=1.25, z=1.25))  # Default camera position
    if camera_position:
        camera['eye'] = camera_position

    fig.update_layout(scene=dict( xaxis=dict(nticks=4, range=x_range),
                                  yaxis=dict(nticks=4, range=y_range),
                                  zaxis=dict(nticks=4, range=z_range),
                                 ),
                      width=width,
                      margin=dict(r=2, l=1, b=1, t=1),
                      scene_camera=camera,
                      showlegend=False,
                     )
    return fig

In [3]:
def convert_2D_to_3D( v ):
    ''' convert a 2D vector (x,y) to a 3d vector (x,y,0) in the x-y plane'''
    if len(v) == 2: return np.array( [v[0], v[1], 0])
    return np.array(v)

def is_number_not_iterable(variable):
    # convenience routine to distinguish scalars from vectors and matrices
    from collections.abc import Iterable
    import numbers
    # Check if the variable is a number
    if isinstance(variable, numbers.Number):
        # Check if the variable is not an iterable
        if not isinstance(variable, Iterable):
            return True
    return False

def line_segment_trace(start, end, color='blue', width=6):
    '''draw a line segment from start_point to end_point of given width and color (converts to 3D)'''
    start_point = convert_2D_to_3D( start )
    end_point   = convert_2D_to_3D( end )

    # Extract the coordinates of the start and end points
    x_coords = [start_point[0], end_point[0]]
    y_coords = [start_point[1], end_point[1]]
    z_coords = [start_point[2], end_point[2]]

    # Create the 3D line segment
    line = go.Scatter3d(
        x=x_coords,
        y=y_coords,
        z=z_coords,
        mode='lines',  # Define the mode as lines to draw a line segment
        line=dict(
            width=width,  # Set the line width
            color=color,  # Set the line color
        )
    )
    return [line]

def cone_trace(base_center, tip_location, base_radius, color='blue'):
    '''draw a cone from base_center to tip_location of given base_radius and color: note 3D vectors'''

    # Calculate the vector components from base to tip
    u = tip_location[0] - base_center[0]
    v = tip_location[1] - base_center[1]
    w = tip_location[2] - base_center[2]

    # Calculate the magnitude of the vector for normalization
    magnitude = np.sqrt(u**2 + v**2 + w**2)

    # Normalize the vector components to use as direction
    u /= magnitude
    v /= magnitude
    w /= magnitude

    # Create the cone trace
    cone = go.Cone(
        x = [base_center[0]], y = [base_center[1]],  z = [base_center[2]],
        u=[u], v=[v], w=[w],
        colorscale=[[0, color], [1, color]], showlegend=False, showscale=False,
        sizemode="absolute",
        sizeref=2*base_radius*magnitude,  # Adjust the cone size based on the base radius and vector magnitude
        anchor="center"                      # Anchor the cone at the base, tip or center
    )
    return [cone]

def vector_trace( start, end, arrow_length, arrow_size, width=6, color='blue'):
    '''draw a vector from start to end with an arrow of given length and base_radius "arrow_size" (converts to 3D)'''
    start = convert_2D_to_3D( start )
    end   = convert_2D_to_3D( end )


    dir_vec   = end - start
    base      = end - (arrow_length / np.linalg.norm( dir_vec )) * dir_vec
    cone      = cone_trace( base, end, arrow_size, color=color)
    line      = line_segment_trace( start, end-0.8*(end-base), color=color, width=width )
    return line+cone

def plane_from_basis_vectors_trace( vec1, vec2, x_range=(-10, 10), y_range=(-10, 10), plane_color='lightgray' ):
    '''create a plane given 2D or 3D vectors vec1 and vec2
    Remark: cannot create a plane containing the z axis due to (x,y) meshgrid
    '''

    # Create a meshgrid for the plane
    #xx, yy        = np.meshgrid(range(x_range[0],x_range[1]+1), range(y_range[0],y_range[1]+1))
    xx, yy        = np.meshgrid( np.array(x_range), np.array(y_range))
    normal_vector = np.cross(vec1, vec2)
    f             = 0  if abs(normal_vector[2]) < 1e-10 else 1/normal_vector[2]
    z             = (-normal_vector[0] * xx - normal_vector[1] * yy) *f

    # Define the plane traces
    return [go.Surface(x=xx, y=yy, z=z, opacity=0.9, name='Plane', colorscale=[[0, plane_color], [1, plane_color]], showscale=False)]

In [4]:
def plane_with_basis_vectors_trace( vec1, vec2, scale=1,
                                arrow_length=2, arrow_size=0.3, width=6, vector_color='blue',
                                x_range=(-10, 10), y_range=(-10, 10), plane_color='lightgray' ):
    '''create a plane given 2D or 3D vectors vec1 and vec2, together with the vectors
    Remark: cannot create a plane containing the z axis due to (x,y) meshgrid
    '''
    traces = []
    traces += plane_from_basis_vectors_trace( vec1, vec2, x_range=x_range, y_range=y_range, plane_color=plane_color)
    o = np.array([0,0,0])
    traces += vector_trace( o, scale*vec1, arrow_length, arrow_size, width=width, color=vector_color )
    traces += vector_trace( o, scale*vec2, arrow_length, arrow_size, width=width, color=vector_color )

    return traces

def ijk_coordinate_vectors_trace(scale=1, arrow_length=2, arrow_size=.3, width=3, color='blue'):
    o  = np.array([0, 0, 0])
    v1 = scale*np.array([1, 0, 0])
    v2 = scale*np.array([0, 1, 0])
    v3 = scale*np.array([0, 0, 1])
    traces = []
    traces += vector_trace( o, v1, arrow_length, arrow_size, width=width, color=color )
    traces += vector_trace( o, v2, arrow_length, arrow_size, width=width, color=color )
    traces += vector_trace( o, v3, arrow_length, arrow_size, width=width, color=color )
    return traces

## 1.2 Example

In [5]:
# Example
traces = []
traces += ijk_coordinate_vectors_trace(scale=10, color='black')
traces += plane_with_basis_vectors_trace( np.array([1,1,0]), np.array([1,0,0.4]), scale=5,
                 vector_color = "#E9EBDF", plane_color='#989F7A'
)

fig = create_fig(traces, camera_position = dict(x=1.4, y=1., z=0.3))
pn.Row( pn.pane.Plotly(fig))

Row
    [0] Plotly(Figure)

# 2. Fundamental Theorem Example

In [6]:
A = np.array([[ 1, -0.2, 1 ]])

R_basis = A[0,:]
N_basis = [ np.array([0.2,1,0]), np.array([-1,0,1]) ]

traces = []
traces += ijk_coordinate_vectors_trace(scale=7, color='gray', arrow_length=1)

# add the null space
traces += plane_with_basis_vectors_trace( *N_basis,
                 vector_color = "#E9EBDF", scale=5, plane_color='#989F7A'
)
# add the row space
traces += vector_trace( np.array([0,0,0]), 4*R_basis, 0.8, 0.7, color="darkred" )
traces += line_segment_trace( -8*R_basis, 8*R_basis, "darkred", 3 )


fig = create_fig(traces, camera_position = dict(x=1.4, y=1., z=0.3))
pn.Row( pn.pane.Plotly(fig))

Row
    [0] Plotly(Figure)